# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint
from sglang.lang.api import set_default_backend
from sglang.srt.utils import load_image
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import print_highlight, terminate_process, wait_for_server

server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

W0831 04:11:55.568000 3893134 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0831 04:11:55.568000 3893134 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-08-31 04:11:56] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=36511, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=Fals

[2025-08-31 04:11:56] Using default HuggingFace chat template with detected content format: string


W0831 04:12:03.949000 3893350 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0831 04:12:03.949000 3893350 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0831 04:12:03.961000 3893351 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0831 04:12:03.961000 3893351 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-08-31 04:12:04] `torch_dtype` is deprecated! Use `dtype` instead!


...

[2025-08-31 04:12:10] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=62.72 GB


[2025-08-31 04:12:11] INFO:     Started server process [3893134]
[2025-08-31 04:12:11] INFO:     Waiting for application startup.
[2025-08-31 04:12:11] INFO:     Application startup complete.
[2025-08-31 04:12:11] INFO:     Uvicorn running on http://0.0.0.0:36511 (Press CTRL+C to quit)


[2025-08-31 04:12:12] INFO:     127.0.0.1:38598 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-31 04:12:12] INFO:     127.0.0.1:38600 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-31 04:12:12] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:13] INFO:     127.0.0.1:38610 - "POST /generate HTTP/1.1" 200 OK
[2025-08-31 04:12:13] The server is fired up and ready to roll!


Server started on http://localhost:36511


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-08-31 04:12:17] INFO:     127.0.0.1:38622 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-08-31 04:12:17] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:17] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.96, #queue-req: 0, 
[2025-08-31 04:12:17] INFO:     127.0.0.1:38636 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-08-31 04:12:17] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:17] INFO:     127.0.0.1:38638 - "POST /generate HTTP/1.1" 200 OK


[2025-08-31 04:12:17] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: True, gen throughput (token/s): 147.64, #queue-req: 0, 
[2025-08-31 04:12:17] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 73, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:18] INFO:     127.0.0.1:38646 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-08-31 04:12:18] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-31 04:12:18] INFO:     127.0.0.1:35928 - "POST /generate HTTP/1.1" 200 OK
[2025-08-31 04:12:18] Prefill batch. #new-seq: 1, #new-token: 2, #cached-token: 31, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-31 04:12:18] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 31, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-31 04:12:18] INFO:     127.0.0.1:35942 - "POST /generate HTTP/1.1" 200 OK
[2025-08-31 04:12:18] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:18] Decode batch. #running-req: 1, #token: 49, token usage: 0.00, cuda graph: True, gen throughput (token/s): 121.78, #queue-req: 0, 


[2025-08-31 04:12:18] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, cuda graph: True, gen throughput (token/s): 164.14, #queue-req: 0, 


[2025-08-31 04:12:18] Decode batch. #running-req: 1, #token: 129, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.05, #queue-req: 0, 


[2025-08-31 04:12:18] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, cuda graph: True, gen throughput (token/s): 162.27, #queue-req: 0, 
[2025-08-31 04:12:18] INFO:     127.0.0.1:35954 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-08-31 04:12:18] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-31 04:12:18] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-31 04:12:19] Decode batch. #running-req: 2, #token: 134, token usage: 0.01, cuda graph: True, gen throughput (token/s): 231.60, #queue-req: 0, 


[2025-08-31 04:12:19] Decode batch. #running-req: 2, #token: 214, token usage: 0.01, cuda graph: True, gen throughput (token/s): 321.14, #queue-req: 0, 


[2025-08-31 04:12:19] Decode batch. #running-req: 2, #token: 294, token usage: 0.01, cuda graph: True, gen throughput (token/s): 317.93, #queue-req: 0, 
[2025-08-31 04:12:19] INFO:     127.0.0.1:35968 - "POST /generate HTTP/1.1" 200 OK


[2025-08-31 04:12:19] INFO:     127.0.0.1:35982 - "POST /generate HTTP/1.1" 200 OK
[2025-08-31 04:12:20] Prefill batch. #new-seq: 1, #new-token: 316, #cached-token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-31 04:12:20] Decode batch. #running-req: 1, #token: 358, token usage: 0.02, cuda graph: True, gen throughput (token/s): 191.66, #queue-req: 0, 


[2025-08-31 04:12:20] Decode batch. #running-req: 1, #token: 398, token usage: 0.02, cuda graph: True, gen throughput (token/s): 161.56, #queue-req: 0, 


[2025-08-31 04:12:20] Decode batch. #running-req: 1, #token: 438, token usage: 0.02, cuda graph: True, gen throughput (token/s): 162.15, #queue-req: 0, 


[2025-08-31 04:12:20] Decode batch. #running-req: 1, #token: 478, token usage: 0.02, cuda graph: True, gen throughput (token/s): 162.12, #queue-req: 0, 


[2025-08-31 04:12:21] INFO:     127.0.0.1:35994 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-08-31 04:12:21] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:21] Decode batch. #running-req: 1, #token: 35, token usage: 0.00, cuda graph: True, gen throughput (token/s): 41.26, #queue-req: 0, 
[2025-08-31 04:12:21] INFO:     127.0.0.1:36008 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-08-31 04:12:21] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:22] Decode batch. #running-req: 1, #token: 68, token usage: 0.00, cuda graph: True, gen throughput (token/s): 82.14, #queue-req: 0, 


[2025-08-31 04:12:23] Decode batch. #running-req: 1, #token: 108, token usage: 0.01, cuda graph: True, gen throughput (token/s): 48.88, #queue-req: 0, 


[2025-08-31 04:12:24] Decode batch. #running-req: 1, #token: 148, token usage: 0.01, cuda graph: True, gen throughput (token/s): 42.11, #queue-req: 0, 
[2025-08-31 04:12:24] INFO:     127.0.0.1:36022 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-08-31 04:12:24] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-31 04:12:24] INFO:     127.0.0.1:36030 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 29.33it/s]

100%|██████████| 3/3 [00:00<00:00, 29.08it/s]

[2025-08-31 04:12:24] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-31 04:12:24] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 17, token usage: 0.00, #running-req: 1, #queue-req: 0, 
[2025-08-31 04:12:24] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 19, token usage: 0.00, #running-req: 2, #queue-req: 0, 
[2025-08-31 04:12:24] INFO:     127.0.0.1:36036 - "POST /generate HTTP/1.1" 200 OK
[2025-08-31 04:12:24] INFO:     127.0.0.1:36046 - "POST /generate HTTP/1.1" 200 OK
[2025-08-31 04:12:24] INFO:     127.0.0.1:36032 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-08-31 04:12:24] INFO:     127.0.0.1:36050 - "POST /generate HTTP/1.1" 200 OK
[2025-08-31 04:12:24] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, 
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-08-31 04:12:24] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:24] INFO:     127.0.0.1:36058 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

W0831 04:12:31.104000 3894192 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0831 04:12:31.104000 3894192 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-08-31 04:12:32] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30151, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7835956249999999, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1,

[2025-08-31 04:12:34] Using default HuggingFace chat template with detected content format: openai


W0831 04:12:40.469000 3894406 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0831 04:12:40.469000 3894406 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0831 04:12:40.580000 3894405 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0831 04:12:40.580000 3894405 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-08-31 04:12:41] `torch_dtype` is deprecated! Use `dtype` instead!


...

[2025-08-31 04:12:55] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=128000, available_gpu_mem=60.78 GB


[2025-08-31 04:12:56] INFO:     Started server process [3894192]
[2025-08-31 04:12:56] INFO:     Waiting for application startup.
[2025-08-31 04:12:56] INFO:     Application startup complete.
[2025-08-31 04:12:56] INFO:     Uvicorn running on http://0.0.0.0:30151 (Press CTRL+C to quit)


[2025-08-31 04:12:57] INFO:     127.0.0.1:33470 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-31 04:12:57] INFO:     127.0.0.1:33484 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-31 04:12:57] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:12:59] INFO:     127.0.0.1:33496 - "POST /generate HTTP/1.1" 200 OK
[2025-08-31 04:12:59] The server is fired up and ready to roll!


Server started on http://localhost:30151


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-08-31 04:13:02] INFO:     127.0.0.1:44610 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-08-31 04:13:03] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-31 04:13:04] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: True, gen throughput (token/s): 4.81, #queue-req: 0, 


[2025-08-31 04:13:04] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: True, gen throughput (token/s): 148.96, #queue-req: 0, 
[2025-08-31 04:13:04] INFO:     127.0.0.1:44614 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)